# Livrable 1 : Classification Binaire
|Erwan Martin|Thibaut Liger-Hellard|Arnaud Maturel|Guillaume Le Cocguen|Victorien Goudeau|
|------------|---------------------|--------------|--------------------|-----------------|

**Contexte**:
L'objectif de ce livrable est classifier les photos des "non-photos" parmis les images du dataset.
La classification sera donc binaire.


# import des librairies

In [1]:
import os
from random import Random, random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import PIL
import random


In [2]:
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NVIDIA GeForce RTX 2060


# Préparation des données
Afin de faire la classification binaire des images il va falloir préparer nos données en étiquettant nos images et en les mettant a la même échelle (scaling).

In [3]:
#data path definition
f = open('../data/path.txt', "r")
DATAPATH = f.read()
print(DATAPATH)

D:\CESI\A5\datascience\Projet\data


In [4]:
# dataset creation
labels = []
imgs = []

#creating labels table -----------------------------
for i in os.listdir(path=DATAPATH):
    if os.path.isdir(DATAPATH+"/"+i):
        labels.append(i)
#end creating labels table -----------------------------

#removing livrable2  from dataset------------------------
labels.remove("Dataset_L2")
labels.remove("Photo_2")
#end removing livrable2 from dataset------------------------


print(f"LABELS : {labels}")

#create BIG csv with image path + label
csv = open("../data/dataset_L1.csv", "w")
csv.write("pathname;label\n")
for label in labels:
    if label == "Photo":
        l = 1
    else:
        l=0
    for img in os.listdir(path=DATAPATH+"/"+label):
        csv.write(DATAPATH+"/"+label+"/"+img+";"+str(l)+"\n")
csv.close()

LABELS : ['Painting', 'Photo', 'Schematics', 'Sketch', 'Text']


maintenant, il faut le load dans un pandas, regardons ce que ca donne:

In [5]:
data = pd.read_csv("../data/dataset_L1.csv", sep=';')
data.head()

,pathname,label
0,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
1,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
2,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
3,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
4,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0


In [9]:
img_size = 400

class ImagesDataset(Dataset):
    def __init__(self, path, file_path, labels, transform = None):
        super().__init__()
        self.df = pd.read_csv(file_path, sep=";", header=0)
        self.transform = transform
        self.class2index = {}
        for i in range(len(labels)):
            self.class2index.update({labels[i]:i})
            #{"label0":0 , "label1":1,...}
        

    def __getitem__(self, index):
        filename = self.df.loc[index]["pathname"]
        label = self.df.loc[index]["label"]
        image = PIL.Image.open(filename)
        if self.transform is not None:
            image = self.transform(image)
        return image, label


    def __len__(self):
        return len(self.df)
    


#load into batched data
batch_size = 100

#transforms if we need
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    torchvision.transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
    
])


full_dataset = ImagesDataset(DATAPATH, "../data/dataset_L1.csv", ["0","1"], transform=transform)

print(len(full_dataset))

#split
generator = torch.Generator().manual_seed(42)
train_set, test_set = random_split(full_dataset, [0.7, 0.3], generator=generator)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)


print(f"TRAIN : {len((train_loader))}")
print(f"TEST : {len(test_loader)}")


41399
TRAIN : 290
TEST : 125


# CNN

In [10]:
class CNN(nn.Module):
    def __init__(self, img_witdh, num_labels):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)#3 color channels #6 outputs channels, 5 kernel size
        outputsize = self.get_outputSize(img_witdh, 5, 1 ,0)

        self.pool = nn.MaxPool2d(2,2) #kernel size 2, stride 2
        outputsize = self.get_outputSize(outputsize, 2, 2 ,0)

        self.conv2 = nn.Conv2d(6, 16, 5)#last output size from the convolution in input
        outputsize = self.get_outputSize(outputsize, 5, 1 ,0)
        
        self.pool = nn.MaxPool2d(2,2) #kernel size 2, stride 2
        outputsize = self.get_outputSize(outputsize, 2, 2 ,0)

        self.afterconv_size = 16*outputsize*outputsize

        self.fc1 = nn.Linear(self.afterconv_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_labels) 

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        #print("1 OK")
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, self.afterconv_size) #flatten the tensor
        #print("3 OK")
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def get_outputSize(self, width_in, kernelSize, stride, padding):
        return int(((width_in - kernelSize + 2*padding  )/stride)+1)

# Training

In [11]:
model = CNN(img_witdh=400, num_labels=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

n_total_steps = len(train_loader)

num_epochs = 10


#Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if (i+1)%len(train_loader) == 0: #pseudos callbacks a chaque epoch
        print(f'epoch : {epoch+1}/{num_epochs}, step : {i+1}/{n_total_steps}, loss = {loss.item():.4f} ') 

        # if (i+1)%2000 == 0:
        #     print(f'epoch : {epoch+1}/{num_epochs}, step : {i+1}/{n_total_steps}, loss = {loss.item():.4f} ')

epoch : 1/10, step : 1/290, loss = 0.6879 
epoch : 1/10, step : 2/290, loss = 2.1380 
epoch : 1/10, step : 3/290, loss = 0.4411 
epoch : 1/10, step : 4/290, loss = 0.8877 
epoch : 1/10, step : 5/290, loss = 0.5974 
epoch : 1/10, step : 6/290, loss = 0.4521 
epoch : 1/10, step : 7/290, loss = 0.4949 
epoch : 1/10, step : 8/290, loss = 0.3923 
epoch : 1/10, step : 9/290, loss = 0.4257 
epoch : 1/10, step : 10/290, loss = 0.4313 
epoch : 1/10, step : 11/290, loss = 0.3738 
epoch : 1/10, step : 12/290, loss = 0.4250 
epoch : 1/10, step : 13/290, loss = 0.4327 
epoch : 1/10, step : 14/290, loss = 0.3541 
epoch : 1/10, step : 15/290, loss = 0.3795 
epoch : 1/10, step : 16/290, loss = 0.4357 
epoch : 1/10, step : 17/290, loss = 0.4101 
epoch : 1/10, step : 18/290, loss = 0.4561 
epoch : 1/10, step : 19/290, loss = 0.4790 
epoch : 1/10, step : 20/290, loss = 0.3907 
epoch : 1/10, step : 21/290, loss = 0.3323 
epoch : 1/10, step : 22/290, loss = 0.4510 
epoch : 1/10, step : 23/290, loss = 0.441

In [12]:
#saving model
torch.save(model.state_dict(), "./pytorch_cnn.pth")

In [14]:
def evaluate_model(model, criterion, test_loader):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(2)]
        n_class_samples = [0 for i in range(2)]
        losses = []

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            #max_returns
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()


            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if(label==pred):
                    n_class_correct[label] +=1
                n_class_samples[label] +=1

            losses.append(criterion(outputs, labels).item())

        mloss = np.mean(losses)
        acc =  100 * n_correct / n_samples
        # print(f'accuracy : {acc}%')
        # print(f'loss : {mloss}%')
        return({"accuracy":acc, "loss":mloss})

evaluate_model(model, criterion, test_loader)

{'accuracy': 86.42402769949271, 'loss': 0.642160052895546}

In [17]:
print(f"TAILLE TEST : {len(test_loader)*batch_size}")

TAILLE TEST : 12500
